## Q. 기상 실측 데이터를 토대로 태양광 발전량인 amount(2023-10-15) 값을 예측해보시오.

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from torch.utils.data import TensorDataset, DataLoader

pd.set_option('display.max_column', 20)
pd.set_option('display.max_row', 100)

data = pd.read_csv('./energy_tobigs_question.csv')
data['time'] = pd.to_datetime(data['time'])

# 피처와 타겟 변수 설정
features = ['cloud', 'temp', 'humidity', 'ground_press', 'wind_speed', 'wind_dir', 'rain', 'snow', 'dew_point', 'vis', 'uv_idx', 'azimuth', 'elevation']
target = 'amount'

# 데이터셋 분리 (2023-10-14까지 학습, 2023-10-15 하루 예측)
train_data = data[data['time'] < '2023-10-15'].copy()
test_data = data[(data['time'] >= '2023-10-15') & (data['time'] < '2023-10-16')].copy()

# 학습 데이터에 NaN 값 제거
train_data = train_data.dropna()

# 데이터 스케일링
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

train_data.loc[:, features] = scaler_x.fit_transform(train_data[features])
train_data.loc[:, target] = scaler_y.fit_transform(train_data[[target]])

test_data.loc[:, features] = scaler_x.transform(test_data[features])

### 시퀀스 데이터 생성 함수 작성

In [28]:
# 시퀀스 데이터 생성 함수 수정
# dataY: 각 시퀀스에 대응하는 24시간의 발전량 값
def build_dataset(time_series, seq_length, target_length=24):
    dataX, dataY = [], []
    for i in range(len(time_series) - seq_length - target_length + 1):
        dataX.append(time_series[i:i + seq_length, :-1])
        dataY.append(time_series[i + seq_length:i + seq_length + target_length, -1])  # Direct Multi-step Forecast Strategy에서 24시간의 발전량 값을 가져오도록 수정하였습니다
    return np.array(dataX), np.array(dataY)

# 학습 데이터 생성
# trainY: 24시간의 발전량 값
trainX, trainY = build_dataset(train_data[features + [target]].values, seq_length)

# 텐서로 변환
trainX_tensor = torch.tensor(trainX)
trainY_tensor = torch.tensor(trainY)

# 텐서데이터셋 생성
train_dataset = TensorDataset(trainX_tensor, trainY_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


### 학습한 시계열 모델을 사용해서 예측을 해보세요 (RNN, LSTM, Transformer, Informer 중에서 선택적으로 활용)

In [5]:
# 슬라이딩 윈도우 방식으로 2023-10-15의 24시간 발전량 예측해보기 (Direct Multi-step Forecast Strategy 혹은 Recursive Multi-step Forecast 등 이외의 방법론도 자유롭게 사용 가능)

In [29]:
# 하이퍼파라미터 설정
input_size = trainX.shape[2]  # 입력 feature의 개수
hidden_size = 64  # hidden state 크기
output_size = 24  # 예측할 값 (24시간 발전량을 한 번에 예측)
num_layers = 2  # RNN 레이어 수
seq_length = 24  # 시퀀스 길이
batch_size = 32  # 배치 사이즈 설정

# RNN 모델 생성
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # 24개의 출력을 위한 Linear Layer

    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)  # 초기 hidden state
        out, hn = self.rnn(x, h0)  # RNN Forward Pass
        out = self.fc(out[:, -1, :])  # 마지막 시퀀스의 hidden state로부터 24개의 출력 생성
        return out

# 모델 초기화
model = RNNModel(input_size, hidden_size, output_size, num_layers)

# Loss 및 Optimizer 정의
criterion = nn.MSELoss()  # 평균 제곱 오차 (MSE)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 과정
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):  # train_loader에서 데이터를 가져옴
        inputs, labels = inputs.to(torch.float32), labels.to(torch.float32)

        # 예측 수행
        predictions = model(inputs)

        # Loss 계산
        loss = criterion(predictions, labels)  # labels는 24시간 발전량을 포함

        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

print("모델 학습 완료")

Epoch [10/50], Loss: 0.0113
Epoch [20/50], Loss: 0.0104
Epoch [30/50], Loss: 0.0091
Epoch [40/50], Loss: 0.0077
Epoch [50/50], Loss: 0.0066
모델 학습 완료


In [30]:
model.eval()
with torch.no_grad():
    # 24시간 예측
    testX = test_data[features].values[-seq_length:]  # 마지막 24시간의 기상 데이터를 사용
    testX_tensor = torch.tensor(testX).unsqueeze(0).to(torch.float32)  # 배치 차원을 추가

    # 예측 수행
    predicted_amounts = model(testX_tensor).cpu().numpy()  # 24시간 예측값

# 예측된 값 역변환 (scaler를 사용하여 실제 값으로 복원)
predicted_amounts = scaler_y.inverse_transform(predicted_amounts).flatten()

# 2023-10-15 예측 발전량 출력
print(f'2023-10-15 예측 발전량 (24시간): {predicted_amounts} kWh')

2023-10-15 예측 발전량 (24시간): [-0.9501733  -3.0736349  -3.610837   -3.5983167  -3.4467387  -3.2091672
 -0.96980053  2.256753   13.165339   35.229153   57.554695   75.44066
 82.99805    83.60589    79.85097    70.420044   48.556076   25.481735
  9.324808    3.5533454   2.3373246   3.9636595   2.9590771   1.1149931 ] kWh


### 2023-10-15 발전량 예측값을 predicted_amounts에 저장하고 시각화를 실행해주세요.

In [33]:
# 시각화
fig = go.Figure()

# 실제 발전량 시각화
fig.add_trace(go.Scatter(x=test_data['time'], y=data['amount'], # 15일에 해당하는 부분만 시각화 하기 위해 x 수정하였습니다
                         mode='lines', name='Actual Amount'))

# 예측된 발전량 시각화 (2023-10-15의 24시간 동안의 예측)
fig.add_trace(go.Scatter(x=test_data['time'], y=predicted_amounts,
                         mode='lines', name='Predicted Amount', line=dict(dash='dot', color='red')))

# 레이아웃 설정
fig.update_layout(title='2023-10-15 24시간 발전량 예측',
                  xaxis_title='시간',
                  yaxis_title='발전량 (kWh)')

# 그래프 출력
fig.show()

### 정답을 적어주세요.

In [34]:
# 예측된 24시간 발전량 출력
print(f'2023-10-15 예측 발전량 (24시간): {predicted_amounts} kWh')

2023-10-15 예측 발전량 (24시간): [-0.9501733  -3.0736349  -3.610837   -3.5983167  -3.4467387  -3.2091672
 -0.96980053  2.256753   13.165339   35.229153   57.554695   75.44066
 82.99805    83.60589    79.85097    70.420044   48.556076   25.481735
  9.324808    3.5533454   2.3373246   3.9636595   2.9590771   1.1149931 ] kWh


---

### (필수) Informer모델은 transformer 모델의 어떤 부분을 개선하고자 했나요? (차이점을 중심으로 서술)

In [8]:
Transformer 모델은 입력 시퀀스가 길어질수록 처리 속도가 급격히 느려지며 메모리 사용량도 증가합니다.
Informer 모델은 이를 개선하기 위해 ProbSparse Self-Attention 메커니즘을 도입했습니다. 이 메커니즘은 중요도가 높은 쿼리(query)에만 집중적으로 계산해 더 효율적인 시계열 예측이 가능합니다.

### (필수) 모델링 해석

In [9]:
# 모델을 선택했다면 왜 선택했는지 본인만의 근거를 정리해주세요.
# 더 나아가 파라미터 선택의 기준이 있었다면 좋습니다.
RNN 모델을 선택한 이유는 시계열 데이터의 시간적 종속성을 학습하는 데 적합하기 때문입니다.
특히 태양광 발전량 예측에서는 시간에 따른 패턴을 잘 반영할 수 있는 RNN 구조가 효율적이며, 복잡한 패턴을 추출하기 위해 다층 RNN 구조를 사용했습니다.

### (선택) 데이터 해석

In [10]:
# 데이터셋을 보고 느낀 생각이나 본인만의 논리 전개 방식을 정리해주세요.
# 전처리를 했다면 해당 전처리를 왜 했는지, 파생변수를 생성했다면 왜 만들었는지 등
태양광 발전량은 주로 기상 데이터에 큰 영향을 받으며, 시간에 따른 변동이 크다는 점이 눈에 띕니다.
특히 일사량, 기온, 습도 등 외부 요인이 시간에 따라 변화하는 패턴을 가지므로, 시계열 모델을 통해 이러한 시간적 패턴을 잘 학습하는 것이 중요하다고 판단했습니다.